# Generacion de instancias

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
DEBUG = False
LOCALES = range(5, 26) # [5, 25]
INSTANCIAS = 3

In [19]:
import os

from typing import List
from typing import Tuple

Local = Tuple[int, int]
def save_instance(dataset: str, instance: str, M: int, locales: List[Local]):
    """
    Guarda una instancia con el formato

        n M
        b_1 c_1
        .
        .
        .
        b_n c_n

    Las instancias son tuplas (b_i, c_i)
    """

    # exist_ok para que no falle si ya existe
    os.makedirs(f"instancias/{dataset}", exist_ok=True)

    if DEBUG: print("Escribiendo instancia", f"{os.getcwd()}/instancias/{dataset}/{instance}.in")
    with open(f"instancias/{dataset}/{instance}.in", "w") as f:
        print(len(locales), M, file=f)
        for local in locales:
            print(local[0], local[1], file=f)

In [20]:
def filename(**kwargs) -> str:
    # 'n-20_i-20_grupo-10_M-20'
    name = ""
    for arg, value in kwargs.items():
        name += f"{arg}_{value}-"
    
    return name[:-1]

## Control

Instancias generadas de forma aleatoria

In [21]:
from typing import Tuple, List
Instance = Tuple[int, List[Local]]

In [22]:
import numpy as np

def randrange(r, n):
    return np.random.randint(low=r[0], high=r[1], size=n)

# rangos
CONTAGIO = (10, 100)
BENEFICIO = (10, 100)

def randinst(n: int) -> Instance:
    locales = zip(
        randrange(BENEFICIO, n),
        randrange(CONTAGIO, n),
    )
    
    m = randrange((CONTAGIO[0] * n, CONTAGIO[1] * n), 1)[0]

    return m, list(locales)
    

In [23]:
for i in range(INSTANCIAS):
    for n in LOCALES:
        m, locales = randinst(n)
        save_instance(dataset="control", instance=filename(n=n, i=i), M=m, locales=locales)

## Grupos - Solapamiento

In [24]:
BENEFICIO = (10, 50)

def group_instance(cant_grupos: int, n: int) -> Instance:
    locales = []
    m = 0
    for l in range(n):
        c = (l % cant_grupos + 1)
        m += c
        b = randrange(BENEFICIO, 1)[0]
        locales.append((b, c))

    return m, locales
    

In [25]:
import random

for i in range(INSTANCIAS):
    for n in LOCALES:
        for g in range(1, n+1):
            # hay una cantidad de grupos n determinada para cada instancia
            m, locales = group_instance(g, n)
            save_instance("grupos", filename(n=n, grupos=g, i=i), m, locales)


## One tu rule dem ol

In [26]:
beneficio = 10
contagio = 10

for i in range(INSTANCIAS):
    for n in LOCALES:
        for p in range(n):
            ben_distinguido = beneficio*n
            m = int(contagio * n / 2)

            locales = [ (beneficio, contagio) for _ in range(n) ]
            locales[p] = (ben_distinguido, contagio)
            save_instance("one-to-rule", filename(n=n, pos=p, i=i), m, locales)


### ¿En qué lugar lo ponemos?

- Podemos variar el lugar en que se coloca el local distinguido.

## Identicos

In [27]:
beneficio = 10
contagio = 10

for i in range(INSTANCIAS):
    for n in LOCALES:
        M = int(contagio * n / 2)
        locales = [ (beneficio, contagio) for _ in range(n) ]
        save_instance("identicos", filename(n=n, i=i), M, locales)


## Factibilidad

In [28]:
BENEFICIO = (10,50)
CONTAGIO = (50,100)
M = 200

def fact_instances(n: int) -> Instance:
    locales = zip(
        randrange(BENEFICIO, n),
        randrange(CONTAGIO, n),
    )

    return M, list(locales)
    

In [29]:
for i in range(INSTANCIAS):
    for n in LOCALES:
        m, locales = fact_instances(n)
        save_instance("low-M", filename(n=n, i=i), m, locales)


## Caché en PD

In [30]:
BENEFICIO = (10,50)
CONTAGIO = (10,500)

def cache_merchants(n: int) -> List[Local]:
    locales = zip(
        randrange(BENEFICIO, n),
        randrange(CONTAGIO, n),
    )

    return list(locales)

In [31]:
# Lo importante de esta experimentación es que no exista solapamiento entre los llamados a la memoria para que se utilicen
# diferentes regiones de la tabla y esto lleve a que se termine desalojando de a una página de memoria (idealmente)  

M = range(1000, 5000, 1000)

for i in range(INSTANCIAS):
    for n in LOCALES:
        for m in M:
            save_instance("cache", filename(n=n, m=m, i=i), m, cache_merchants(n))

## Programación Dinámica

In [32]:
N = range(50, 1050, 50)
M = range(50, 1050, 50)
BENEFICIO = (1, 10)
CONTAGIO = (1, 10)

def randinst(n: int) -> List[Local]:
    locales = zip(
        randrange(BENEFICIO, n),
        randrange(CONTAGIO, n),
    )

    return list(locales)

for m in M:
    for n in N:
        for i in range(INSTANCIAS):
            save_instance("complejidad-DP", filename(n=n, m=m, i=i), m, randinst(n))